In [1]:
from dataclasses import dataclass, field
from typing import List, Set, Optional

In [2]:
@dataclass
class ValueLimit:
    lower: int
    upper: int

In [3]:
@dataclass
class Field:
    name: str
    limits: List[ValueLimit]
    def matches(self, value):
        for limit in self.limits:
            if limit.lower <= value <= limit.upper:
                return True
        return False
    def __hash__(self):
        return hash(self.name)

In [4]:
@dataclass
class TicketValue:
    value: int
    possible_fields: Set[Field] = field(default_factory=lambda: set())
    def match_fields(self, fields: List[Field]):
        self.possible_fields = set(f for f in fields if f.matches(self.value))

In [5]:
@dataclass
class Ticket:
    values: List[TicketValue]

In [6]:
def get_input(fname="input.txt"):
    fields: List[Field] = []
    ticket: Optional[Ticket] = None
    other_tickets: List[Ticket] = []
    with open(fname) as f:
        lines = iter(f.readlines())
        line = next(lines).strip()
        while line != "":
            name, llist = line.split(": ")
            llist = llist.split(" or ")
            limits: List[ValueLimit] = []
            for l in llist:
                parts = l.split("-")
                limits.append(ValueLimit(int(parts[0]), int(parts[1])))
            fields.append(Field(name, limits))
            line = next(lines).strip()
        next(lines)
        line = next(lines).strip()
        ticket = Ticket([TicketValue(int(v)) for v in line.split(",")])
        next(lines)
        next(lines)
        line = next(lines)
        while line is not None:
            other_tickets.append(Ticket([TicketValue(int(v)) for v in line.strip().split(",")]))
            line = next(lines, None)
        return fields, ticket, other_tickets

In [7]:
test_fields, test_ticket, test_other_tickets = get_input("test.txt")

In [8]:
test_fields

[Field(name='class', limits=[ValueLimit(lower=1, upper=3), ValueLimit(lower=5, upper=7)]),
 Field(name='row', limits=[ValueLimit(lower=6, upper=11), ValueLimit(lower=33, upper=44)]),
 Field(name='seat', limits=[ValueLimit(lower=13, upper=40), ValueLimit(lower=45, upper=50)])]

In [9]:
test_ticket

Ticket(values=[TicketValue(value=7, possible_fields=set()), TicketValue(value=1, possible_fields=set()), TicketValue(value=14, possible_fields=set())])

In [10]:
test_other_tickets

[Ticket(values=[TicketValue(value=7, possible_fields=set()), TicketValue(value=3, possible_fields=set()), TicketValue(value=47, possible_fields=set())]),
 Ticket(values=[TicketValue(value=40, possible_fields=set()), TicketValue(value=4, possible_fields=set()), TicketValue(value=50, possible_fields=set())]),
 Ticket(values=[TicketValue(value=55, possible_fields=set()), TicketValue(value=2, possible_fields=set()), TicketValue(value=20, possible_fields=set())]),
 Ticket(values=[TicketValue(value=38, possible_fields=set()), TicketValue(value=6, possible_fields=set()), TicketValue(value=12, possible_fields=set())])]

In [11]:
test_ticket.values[0].match_fields(test_fields)

In [12]:
test_ticket.values[0].possible_fields

{Field(name='class', limits=[ValueLimit(lower=1, upper=3), ValueLimit(lower=5, upper=7)]),
 Field(name='row', limits=[ValueLimit(lower=6, upper=11), ValueLimit(lower=33, upper=44)])}

In [13]:
def find_invalid_tickets(tickets, fields):
    invalid_values = []
    invalid_tickets = []
    for ticket in tickets:
        invalid = False
        for value in ticket.values:
            value.match_fields(fields)
            if len(value.possible_fields) == 0:
                invalid_values.append(value)
                invalid = True
        if invalid:
            invalid_tickets.append(ticket)
    return invalid_tickets, invalid_values

In [14]:
sum(v.value for v in find_invalid_tickets(test_other_tickets, test_fields)[1])

71

In [15]:
fields, ticket, other_tickets = get_input("input.txt")

In [16]:
sum(v.value for v in find_invalid_tickets(other_tickets, fields)[1])

29878

In [17]:
def find_fields_mapping(tickets, fields):
    invalid = find_invalid_tickets(tickets, fields)[0]
    mappings = [None for _ in range(len(tickets[0].values))]
    possible_values = [set(value.possible_fields) for value in tickets[0].values]
    for ticket in tickets:
        if ticket in invalid:
            continue
        for i, v in enumerate(ticket.values):
            possible_values[i] &= v.possible_fields
    mapped = set()
    remaining = set(range(len(mappings)))
    while len(remaining) > 0:
        for i in list(remaining):
            fields = possible_values[i]
            possible = fields - mapped
            if len(possible) == 1:
                mappings[i] = possible.pop()
                mapped.add(mappings[i])
                remaining.remove(i)
    return mappings

In [18]:
find_fields_mapping([test_ticket] + test_other_tickets, test_fields)

[Field(name='row', limits=[ValueLimit(lower=6, upper=11), ValueLimit(lower=33, upper=44)]),
 Field(name='class', limits=[ValueLimit(lower=1, upper=3), ValueLimit(lower=5, upper=7)]),
 Field(name='seat', limits=[ValueLimit(lower=13, upper=40), ValueLimit(lower=45, upper=50)])]

In [19]:
mappings = find_fields_mapping([ticket] + other_tickets, fields)

In [20]:
result = 1
for i, fld in enumerate(mappings):
    if 'departure' in fld.name:
        result *= ticket.values[i].value

In [21]:
result

855438643439